**Overview Stage 2**

**1. Import data**  
3 different datasets are imported and formatted to comply with input required for the Optimization.
    1. Monthly retail sales data (EIA-861M)
    2. Hourly demand data (EIA-930)
    3. Results of Stage 1
We use retail sales and demand data for 2 years (Oct 2018-2020) as demand data is available starting from October 2018.    

**2. Optimization**    
Using all of the above input data, the code optimizes for a distribution key for subregional demand, using the retail sales and demand data as inputs and the results of Stage 1 as constraints.    

**3. Create hourly demand data profiles based on optimization outputs**   
Use the optimization output to create hourly demand profiles for 2019 for each state.



# Import data

In [2]:
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp
import glob

## import monthly state retail sales data (EIA-861M)

In [58]:
#import montly retail sales for each state
monthly_RS = pd.read_excel(r"C:\Users\Muriel Hauser\Documents\Carnegie Science\State demand data\multivariate linear regression\sales_revenue-1990-current.xlsx", usecols=("A,B,C,Z"), header=2)
#select only period of Oct 2018 until Oct 2020 (this corresponds to the demand data we have available)
monthly_RS_2yrs=monthly_RS.loc[17595:18818]
#monthly_RS_2yrs

,Year,Month,State,Megawatthours.5
17595,2018,10.0,AK,469971.37
17596,2018,10.0,AL,7217313.80
17597,2018,10.0,AR,4046952.10
17598,2018,10.0,AZ,5801700.30
17599,2018,10.0,CA,22425636.00
...,...,...,...,...
18814,2020,9.0,VT,416023.97
18815,2020,9.0,WA,6175347.10
18816,2020,9.0,WI,5304831.40
18817,2020,9.0,WV,2432659.50


##### calculate monthly retail sales by state 

In [93]:
#create dictionary, each key representing one state with 24 values per state representing monthly sums of retail sales from Oct 2018 to 2020
#list of all states in CONUS 
states=monthly_RS_2yrs.State.unique()
states=states.tolist()
#remove states not in CONUS (still including DC as separate state)
states.remove('AK')
states.remove('HI')
states=sorted(states)#create new dictionary
retail_sales_CONUS={}
for i in states:
    #print(i)
    retail_sales_CONUS["state_{0}".format(i)] = monthly_RS_2yrs[monthly_RS_2yrs["State"]==i]['Megawatthours.5'].to_numpy().astype('float')
#retail_sales_CONUS

In [94]:
#add Washington DC to Maryland
retail_sales_CONUS['state_MD']=retail_sales_CONUS['state_MD']+retail_sales_CONUS['state_DC']
#remove DC from the dictionary
retail_sales_CONUS.pop('state_DC', None)
#remove DC from the list of states as it is now treated as part of Maryland
states.remove('DC')

## import hourly demand data (EIA 930)

Import hourly EIA demand data from October 2018 - October 2020 here: https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data, as described here: https://www.nature.com/articles/s41597-020-0483-x 

In [95]:
#function to import demand data
def importfiles(path, dicname):
    for f in path:
        key = f.split('\\')[-1].split('.')[0]
        BA_demand=pd.read_csv(f, error_bad_lines=False)
        #BA_demand=BA_demand.iloc[2208:10968]    #only year 2019
        #BA_demand=BA_demand.iloc[2209:10969]    #only year 2019 for MEM format
        BA_demand=BA_demand.set_index('date_time')
        BA_demand.index=pd.to_datetime(BA_demand.index)
        #BA_demand= BA_demand.resample('M').sum() #aggregate the demand data to monthly values
        #print(key)
        #BA = analyze(f)
        dicname[(key)] = BA_demand


In [96]:
#import demand data and safe as dictionary (each key = one BA)
path=glob.glob(r"C:\Users\Muriel Hauser\Documents\Carnegie Science\State demand data\BA demand data\release_2020_Oct_include_subregions\subregions_and_balancing_authorities\*.csv") #adjust path to your location
demand_data=dict()
importfiles(path,demand_data)
demand_data['PJM-AE'].head() #example

,raw demand (MW),category,cleaned demand (MW),forecast demand (MW)
date_time,,,,
2018-10-01 00:00:00,1323,OKAY,1323,1323
2018-10-01 01:00:00,1279,OKAY,1279,1279
2018-10-01 02:00:00,1199,OKAY,1199,1199
2018-10-01 03:00:00,1106,OKAY,1106,1106
2018-10-01 04:00:00,1013,OKAY,1013,1013


CISO and NYIS subregions are added up to their respective BAs as these subregions comply with state boundaries.


In [97]:
### functions to help sum up subregion demand data to BA level ###
#find BA names for each subregions
def search_subregions(dictionary, BA):
    names=[]
    for v in dictionary.keys():
        #print(v)
        if BA in v:
            #print(v)
            names.append(v)
    return names

#sum the demand of the subregions
def sum_subregions(names_of_subregions, dictionary):
    df_temp=pd.DataFrame()
    for i in names_of_subregions:
        df_temp[i]=dictionary[i]['cleaned demand (MW)']
    df_temp['cleaned demand (MW)']=df_temp.sum(axis=1)
    return df_temp

In [98]:
#sum up subregion demand to BA level
subregions_to_BA_list=['NYIS','CISO']

#for each BA, sum subregions using the above functions
for BA in subregions_to_BA_list:
    names = search_subregions(demand_data,BA)
    demand_data[BA] =sum_subregions(names, demand_data)
    for i in names:
        del demand_data[i]

There is demand data for CPLW, but CPLW is not covered in the utility customers data.Therefore I add CPLW to CPLE.              
(CPLE=Duke Energy Progress East, CPLW=Duke Energy Progress West)

In [99]:
#add CPLW demand data to CPLE
demand_data['CPLE']['cleaned demand (MW)']=demand_data['CPLE']['cleaned demand (MW)']+demand_data['CPLW']['cleaned demand (MW)']
del demand_data['CPLW']

Create one dataframe for all BAs monthly sums.

In [100]:
#create one DF where each column represents the monthly demand for one BA or subregion if available
SubR_demand=pd.DataFrame()
for i in demand_data.keys():
    SubR_demand[i]=demand_data[i]['cleaned demand (MW)']
#aggregate data to monthly sums
SubR_demand_monthly= SubR_demand.resample('M').sum()
#sort columns alphabetically
SubR_demand_monthly = SubR_demand_monthly.sort_index(axis=1)

## import subregion to state distribution matrix 

This matrix can be found on the github repository. It shows which subregions serve which states. The data is based on:  
https://www.pjm.com/library/~/media/about-pjm/pjm-zones.ashx for PJM      
https://api.misoenergy.org/MISORTWD/lmpcontourmap.html for MISO        
https://www.spp.org/documents/16187/im_state_outreach_presentation_apsc.pdf for SWPP

In [101]:
##import data as dataframe
BA_to_state = pd.read_excel(r"C:\Users\Muriel Hauser\Documents\Carnegie Science\State demand data\multivariate linear regression\PJM subregions.xlsx", sheet_name='Sheet4',header=0, index_col=0) #adjust path to your location

##clean up dataframe
#drop the column with written out state names
BA_to_state=BA_to_state.drop('states',axis=1) 
#sort columns alphabetically
BA_to_state = BA_to_state.sort_index(axis=1)
#sort rows alphabetically
BA_to_state = BA_to_state.sort_index(axis=0)
#delete the last row because it does not represent data of a state
BA_to_state.drop(BA_to_state.tail(1).index,inplace=True)
#BA_to_state

In [102]:
#delete subregions for which there is no demand data
del BA_to_state['OVEC'] #Ohio Valley Electric Corporation
del BA_to_state['SEC']  #Florida
del BA_to_state['NBSO'] #New Brunswick System Operator, Canada
del BA_to_state['SEPA'] #Georgia
#change all NaN to 0
BA_to_state=BA_to_state.fillna(0)
#adjust format as required for the optimization (same format as x ([99 BAs : 49 states]))
BA_to_state=BA_to_state.T

## import results of Stage 1

In [103]:
#import the distribution key of Stage 1 
S1_results = pd.read_excel(r"C:\Users\Muriel Hauser\Documents\Carnegie Science\State demand data\fraction_By_state_BAcode.xlsx", index_col=[0,1], header=0) #adjust path to your location

##### create a dataframe that lists the demand per BA and State in a matrix (BA_code x State) as calculated inStage 1

In [104]:
#create df with only distr key in it
distr_key_S1=pd.DataFrame(S1_results['fraction'])
distr_key_S1=pd.DataFrame(distr_key_S1.unstack())
#delete 1st level index of columns
distr_key_S1.columns = distr_key_S1.columns.droplevel()

In [105]:
#delete states outside of CONUS
del distr_key_S1['DC'] #washington dc
del distr_key_S1['AK'] #alaska
del distr_key_S1['HI'] #hawaii
#delete BAs that serve these states +  BA's for which there is no demand data 
distr_key_S1=distr_key_S1.drop('AMPL') #alaska
distr_key_S1=distr_key_S1.drop('HECO') #Hawaiian electric company
distr_key_S1=distr_key_S1.drop('CEA') #Canadian Electricity Association
distr_key_S1=distr_key_S1.drop('SEPA') #Georiga
distr_key_S1=distr_key_S1.drop('OVEC') #Ohio Valley Electric Corporation
distr_key_S1=distr_key_S1.drop('NBSO') #New Brunswick System Operator, Canada
distr_key_S1=distr_key_S1.drop('SEC') #Florida

In [106]:
#add correct subregional names to dataframe
for i in ['ISNE-4003','ISNE-4004', 'ISNE-4007', 'ISNE-4005', 'ISNE-4008', 'ISNE-4006','ISNE-4002','ISNE-4001','PJM-AP','PJM-AEP','PJM-ATSI','PJM-AE','PJM-BC','PJM-CE','PJM-DAY','PJM-DPL','PJM-DOM','PJM-DEOK','PJM-DUQ','PJM-EKPC','PJM-JC','PJM-ME','PJM-PE','PJM-PN','PJM-PL','PJM-PEP','PJM-PS','MISO-0001','MISO-0027','MISO-0035','MISO-0004','MISO-0006','MISO-8910','SWPP-CSWS','SWPP-EDE','SWPP-GRDA','SWPP-INDN','SWPP-KACY','SWPP-KCPL','SWPP-LES','SWPP-MPS','SWPP-NPPD','SWPP-OKGE','SWPP-OPPD','SWPP-SECI','SWPP-SPRM','SWPP-SPS','SWPP-WAUE','SWPP-WFEC','SWPP-WR']:
          distr_key_S1=distr_key_S1.append(pd.Series(name=i,dtype='float64'))

In [107]:
#drop aggregates of subregions
distr_key_S1=distr_key_S1.drop('SWPP')
distr_key_S1=distr_key_S1.drop('PJM')
distr_key_S1=distr_key_S1.drop('MISO')
distr_key_S1=distr_key_S1.drop('ISNE')

In [108]:
#change all NaN to 0
distr_key_S1=distr_key_S1.fillna(0)
#sort columns alphabetically
distr_key_S1 =distr_key_S1.sort_index(axis=1)
#sort rows alphabetically
distr_key_S1 = distr_key_S1.sort_index(axis=0)

# Optimization

We have

$$
\text{min}\ \| Ax -b \|_2 = \sum_j \sqrt{\sum_i (Ax_{:j} - b_{:j})_i^2 }\\
x \in [0, 1]
$$

with the dimensions $A\in\mathcal{R}^{24 x 19}$, $b\in \mathcal{R}^{24 x n_{states}}$ and  $x \in \mathcal{R}^{19 x n_{states}}$.  

You should build $b$ such that $b = [b_1, b_2, ...]$. 


In [109]:
#create A -> 24 month demand for each subregion (19 subregions)
A=SubR_demand_monthly.to_numpy().astype('float')

In [110]:
#create b -> retail sales for 24 months for each state (24x48)
b_total = np.zeros((24, 48))
for i,y in zip(states,range(0,48)):
    b_total[:, y] = retail_sales_CONUS['state_'+str(i)]

In [111]:
#create x
n = 99 #number of BAs and subregions
m = 48 #number of states (without Hawaii and Alaska, Washington DC)
x = cp.Variable((n, m))

In [112]:
#linear optimization 

## objective function ##
state_ = [cp.norm(A @ x[:, i] - b_total[:, i]) for i in range(m)]
obj=cp.Minimize(cp.sum(state_))

## constraints ##

#1. factors need to be between 0 and 1
constraint_factor = [x<=1, x>=0]  

#2. each BA (and subregion) needs to add up to 1
constraint_BA = [cp.sum(x[i, :]) == 1 for i in range(n)]

#3. constraints to show which subregions serve which state
constraints_zero = []
#iterate through x (columns, then rows) and set values to 0 if BA/subregion does not supply a particular state
for j in range(n): #99 rows
    for k in range(m):  #48 states
        if BA_to_state.iloc[j,k]==0:
             constraints_zero.append(x[j,k]==0)
                
#4. constraints from Stage 1
#we assume that the distribution key obtained from Stage 1 applies here as well
#the BA to state distribution for all BAs except PJM, SWPP and MISO are taken from Stage 1
constraints_S1=[]
for j in range(n): #99 rows
    for k in range(m): # 48 columns
        if distr_key_S1.iloc[j,k]!=0:
            constraints_S1.append(x[j,k]==distr_key_S1.iloc[j,k])

#add all constraints together
constraints=constraints_S1+constraints_zero+constraint_BA+constraint_factor

##solve ##
prob = cp.Problem(obj, constraints)
prob.solve( verbose=False)
prob.status

'optimal'

In [113]:
#result of optimization
x.value

array([[ 7.69956025e-01, -3.77758499e-11, -4.00110897e-12, ...,
         1.28489633e-10, -5.79770015e-11, -1.71251304e-10],
       [ 4.05189584e-11,  1.93562493e-02, -3.16074309e-11, ...,
         1.10499554e-10, -5.51276807e-11, -1.99578606e-10],
       [ 4.84315111e-11, -7.86119606e-11, -3.18293221e-11, ...,
         1.33864400e-10, -6.40976032e-11, -2.18874101e-10],
       ...,
       [ 5.91802536e-11, -5.28406243e-11, -1.77265114e-11, ...,
         1.36403189e-10, -2.90923244e-11,  2.50927253e-01],
       [ 6.61131368e-11, -2.36950280e-11,  6.05168087e-01, ...,
         1.41218734e-10, -4.91687792e-11, -1.32902071e-10],
       [ 6.51713097e-11, -6.12688522e-11, -6.36735331e-12, ...,
         1.43231075e-10, -3.63929480e-11,  2.47480180e-01]])

In [114]:
#safe results whereby all values of x smaller than 10^-5=0
x_final = x.value
x_final[x_final<10**(-5)] = 0
print(x_final)

[[0.76995602 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01935625 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.25092725]
 [0.         0.         0.60516809 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.24748018]]


# Create hourly demand profiles based on Stage 1 & 2

In [115]:
#hourly demand for 2019 for all BAs und subregions
SubR_demand_2019=SubR_demand.iloc[2209:10969,:] #choose 2019; MEM format starts and ends one hour later
SubR_demand_2019

,AEC,AECI,AVA,AZPS,BANC,BPAT,CHPD,CPLE,DOPD,DUK,...,TEC,TEPC,TIDC,TPWR,TVA,WACM,WALC,WAUW,NYIS,CISO
date_time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,483,3113,1831,4069,1974,7497,293,6811,247,10811,...,2482,1417,259,732,16079,3710,1051,123,18888,24947
2019-01-01 02:00:00,460,3033,1853,4073,2166,7927,307,6472,261,10287,...,2292,1367,284,774,15438,3659,1070,120,18194,27655
2019-01-01 03:00:00,434,2948,1832,3939,2160,7933,303,6180,260,9796,...,2097,1395,282,770,14967,3579,1047,117,17418,27676
2019-01-01 04:00:00,416,2935,1761,3828,2079,7752,300,5836,257,9294,...,1932,1296,272,748,14381,3485,1034,113,16607,26853
2019-01-01 05:00:00,398,2852,1720,3725,2007,7556,297,5499,251,8804,...,1812,1218,263,720,13865,3375,1003,118,15770,25941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 20:00:00,423,2747,1787,2897,1898,7460,295,5844,257,10157,...,2074,772,254,680,16638,2659,835,104,18288,22014
2019-12-31 21:00:00,419,2680,1771,2858,1843,7442,289,6010,248,10124,...,2060,805,252,681,16264,2610,813,104,18296,21584
2019-12-31 22:00:00,414,2688,1737,2837,1820,7359,294,6506,254,10362,...,2091,810,251,676,16329,2584,790,99,19038,21709


In [116]:
#create df with state demand based on Stage 2 by multiplying BA/subregional demand with distribution key
#create lists with subregion names and state names
subregion_names=list(BA_to_state.index)
state_names=list(BA_to_state.columns)
demand_S2=pd.DataFrame()
for j in state_names:
    temp=pd.DataFrame()
    for i in subregion_names:
        temp[i+j]=distr_key.loc[j][i]*SubR_demand_2019[i]
        #print(temp)
    temp['total']=temp.sum(axis=1)
    demand_S2[j]=temp['total']

In [117]:
#add datetime to demand_S2 index again 
#(for some reason it doesnt recognize the old index as daytime correctly)
rng = pd.date_range('1/1/2019 01:00', periods=8760, freq='1H')
demand_S2['date'] = rng
demand_S2 = demand_S2.set_index('date')
demand_S2.head()

,AL,AR,AZ,CA,CO,CT,DE,FL,GA,IA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,9185.616507,5621.629942,10087.739625,31068.559747,8568.363551,3611.999987,1516.028473,28852.505046,15147.158484,6696.564727,...,2026.701859,11569.674860,47507.892644,4626.025026,13565.336413,696.0,14127.234786,8630.152336,4023.327648,2472.003450
2019-01-01 02:00:00,8851.869451,5572.272984,10073.488996,34347.121915,8367.832813,3428.999988,1444.279536,26656.924373,14625.023989,6517.004980,...,1980.898935,11118.493734,46853.819301,4643.207921,12961.015635,659.0,14793.968411,8362.795981,3869.130371,2470.125564
2019-01-01 03:00:00,8488.406918,5515.962383,9827.054384,34346.314549,8114.141002,3247.999989,1364.558494,24800.458274,13984.077257,6342.385049,...,1927.878296,10756.563462,45988.001005,4584.795299,12334.535603,636.0,14644.146588,8078.034734,3694.902003,2431.950574
2019-01-01 04:00:00,8137.594995,5461.820485,9458.992224,33317.547527,7851.635062,3062.999989,1285.501795,22960.410006,13395.633613,6195.257106,...,1876.485770,10345.276338,45317.916456,4516.280434,11673.618997,603.0,14182.506804,7815.509992,3575.037071,2384.442889
2019-01-01 05:00:00,7789.469785,5359.421298,9136.639324,32212.854095,7568.306989,2885.999990,1211.759832,21060.853250,12790.586448,6039.819881,...,1823.667320,9970.807233,44515.734364,4407.205944,11156.291013,569.0,13813.132374,7524.531137,3435.062716,2322.460488


In [119]:
#safe hourly demand data per state to excel files
for i in state_names:
    state=pd.DataFrame()
    state=demand_S2[i]
    state.index.names = ['date_time'] #change name of index to match MEM input file standards
    state.to_csv(r'C:\\Users\\Muriel Hauser\\Documents\\Carnegie Science\\State demand data\\test\\'+i+'.csv',sep=';',header=['cleaned demand (MW)'],index=True) #adapt path to your location
    

Remember that for States within ISNE, Florida, and California the boundaries of the BA comply wiht state boundaries 
and can therefore be calculated directly